In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import export_text
from sklearn.tree import export_graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
import graphviz

data = pd.read_excel("/content/drive/MyDrive/KPDL/BTL/Top 300 universities of World(5).xlsx")

# Chọn các cột đặc trưng và cột nhãn
features = data[["acceptance_rate", "Average SAT", "Average ACT", "Aid receiving",
                 "Average aid awarded1", "Net average cost to attend1"]]

labels = data["Quality"]

# Chuyển đổi các cột dl dạng văn bãn thành dạng số
features = pd.get_dummies(features)

# Thay thế các kí tự không hợp lệ hoặc đặc biệt thành khoảng trắng
features.columns = features.columns.str.replace('[^a-zA-Z0-9]', ' ', regex=True)

# Tách dl thành 2 tập train và test
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.3, random_state=42)


# Xây dựng mô hình cây quyết định (J48) và  định nghĩa các giá trị them số cần thử nghiệm
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}
# Tạo mô hình Cây quyết định (J48)
clf = DecisionTreeClassifier()
clf.fit(train_features, train_labels)

# Sử dụng GridSearchCV để thữ nghiệm các tham số và lựa chọn mô hình tốt nhất
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(train_features, train_labels)

# Lựa chọn mô hình tit61 nhất sau khi thũ ngiệm
best_clf = grid_search.best_estimator_

# In ra các thông số tốt nhất của mô hình
print("Best Parametres:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

# Tạo biểu diễn đồ thị cho cây quyết dịnh
dot_data = export_graphviz(best_clf, out_file=None, feature_names=list(train_features.columns),
                           class_names=best_clf.classes_, filled=True, rounded=True, special_characters=True)
# Hiển thị đồ thị
graph = graphviz.Source(dot_data)
graph.render(filename="/content/drive/MyDrive/KPDL/BTL/university_tree", format='png', cleanup=True)
graph.view()


# Dự đoán nhãn cho tập kiểm tra
y_pred = clf.predict(test_features)

# Đánh giá mô hình cây quyết định
print("Model Evaluation:")
print(confusion_matrix(test_labels, y_pred))
print(classification_report(test_labels, y_pred))

# Dự đoán hết quả trên tập test
predictions = best_clf.predict(test_features)

# Đánh giá hiệu suất của mô hình trên tập Test
accuracy = accuracy_score(test_labels, predictions)
print('Accuracy on Test Set:', accuracy)

Best Parametres: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}
Best Accuracy: 0.688966318234611
Model Evaluation:
[[31 18]
 [21 20]]
              precision    recall  f1-score   support

     Top Cao       0.60      0.63      0.61        49
    Top Thấp       0.53      0.49      0.51        41

    accuracy                           0.57        90
   macro avg       0.56      0.56      0.56        90
weighted avg       0.56      0.57      0.56        90

Accuracy on Test Set: 0.6666666666666666


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
